In [2]:
# !pip install --upgrade mplfinance
# %pip install statsmodels
# %pip install pmdarima
# %pip install scikit-learn
# %pip install tensorflow



Importing libraries

In [3]:
# Basic libraries
import numpy as np
import pandas as pd
import warnings

import pip
warnings.filterwarnings("ignore") 

# For processing
import math
import random
import datetime as dt
import matplotlib.dates as mdates

# For visualization
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc

# Libraries for model training


from statsmodels.tsa.arima.model import ARIMA

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error

Reading the Data from the database stocks_data (POSTGRES)

In [4]:
import psycopg2
import pandas as pd

# Database credentials
DB_HOST = "localhost"
DB_NAME = "stocks_data"
DB_USER = "postgres"
DB_PASSWORD = "123"
DB_PORT = "5432"

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)

# Query
query = "SELECT * FROM stock_data;"

# Load into pandas DataFrame
df = pd.read_sql(query, conn)

# Close connection
conn.close()

print(df.head())


  companyname stockname       high        low       open      close  \
0  TechnipFMC       FTI  29.855000  29.075001  29.075001  29.615000   
1  TechnipFMC       FTI  30.270000  29.555000  29.610001  30.125000   
2  TechnipFMC       FTI  30.305000  29.434999  30.040001  30.264999   
3  TechnipFMC       FTI  30.219999  29.715000  30.155001  29.915001   
4  TechnipFMC       FTI  30.500000  29.625000  29.830000  30.450001   

      volume        date  
0  1908400.0  2010-01-04  
1  2206600.0  2010-01-05  
2  2724200.0  2010-01-06  
3  2581000.0  2010-01-07  
4  3280400.0  2010-01-08  


In [5]:
df

,companyname,stockname,high,low,open,close,volume,date
0,TechnipFMC,FTI,29.855000,29.075001,29.075001,29.615000,1908400.0,2010-01-04
1,TechnipFMC,FTI,30.270000,29.555000,29.610001,30.125000,2206600.0,2010-01-05
2,TechnipFMC,FTI,30.305000,29.434999,30.040001,30.264999,2724200.0,2010-01-06
3,TechnipFMC,FTI,30.219999,29.715000,30.155001,29.915001,2581000.0,2010-01-07
4,TechnipFMC,FTI,30.500000,29.625000,29.830000,30.450001,3280400.0,2010-01-08
...,...,...,...,...,...,...,...,...
81730,LVMH,MC,53.889999,51.279999,53.689999,51.450001,404700.0,2021-01-26
81731,LVMH,MC,50.779999,48.759998,50.419998,49.369999,688200.0,2021-01-27
81732,LVMH,MC,51.820000,49.639999,49.930000,51.189999,480400.0,2021-01-28
81733,LVMH,MC,51.130001,48.990002,51.099998,49.709999,341000.0,2021-01-29


EXPLOATORY DATA ANALYSIS

unique company names and unique stocknames 
